# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [54]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df_control = df[df['group'] == 'control']
df_experiment = df[df['group'] == 'experiment']

def process_df(df):
    # 1. Encode dummies
    
    df = pd.get_dummies(df,columns=['action'])
    
    # 2. Group ID's
    
    df = df.groupby('id').agg({'action_click':"sum",'action_view':sum})
    
    # 3. Create viewed but not clicked columns
    
    df['click_no_view'] = ((df['action_view'] == 0) & (df['action_click'] == 1))    
    
    # 4. Create clicked and viewed column
    
    df['click_and_view'] = ((df['action_view'] == 1) & (df['action_click'] == 1))
    
    return df

df_cont = process_df(df_control)

df_exp = process_df(df_experiment)

display(df_cont)

display(df_exp)



,action_click,action_view,click_no_view,click_and_view
id,,,,
182994,1,1,False,True
183089,0,1,False,False
183248,1,1,False,True
183515,0,1,False,False
183524,0,1,False,False
...,...,...,...,...
936786,0,1,False,False
937003,0,1,False,False
937073,0,1,False,False


,action_click,action_view,click_no_view,click_and_view
id,,,,
182988,0,1,False,False
183136,0,1,False,False
183141,1,1,False,True
183283,0,1,False,False
183389,0,1,False,False
...,...,...,...,...
935382,0,1,False,False
935576,0,1,False,False
935742,1,1,False,True


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [74]:
result = stats.ttest_ind(df_exp['click_and_view'], df_cont['click_and_view'])
print(result[1])

0.008825098914958293


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [59]:
#Your code here
cont_clicked = np.sum(df_cont['click_and_view'])
cont_mean = cont_clicked/len(df_cont)

exp_clicked = np.sum(df_exp['click_and_view'])
exp_mean = exp_clicked/len(df_exp)



expected_clicks = cont_mean * len(df_exp)
print(expected_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [78]:
#Your code here
import numpy as np
cont_var = len(df_exp)*cont_mean*(1-cont_mean)
cont_std = np.sqrt(cont_var)
z = (exp_clicked - expected_clicks)/cont_std

print(cont_std)
print(z)
print(exp_clicked)
print(cont_clicked)

24.568547907005815
3.6625360854823588
928
932


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [79]:
#Your code here
import scipy.stats as stats
print(1 -stats.norm.cdf(z))



0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.